In [1]:
import os
from Bio import SeqIO
import io
from Bio.Seq import Seq
import subprocess

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length):
    for offset in range(3):
        translated_sequence = str(gene_sequence[offset:].translate()).split("*")
        for i in range(len(translated_sequence)):
            if len(translated_sequence[i])> 0.8*query_length:
                sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
                # print(i+1, offset)
                
                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:
                    output = f"{query_fasta_sequence}\n\n{sequence_set}"
                    out_file.write(output)

In [4]:
def run_mafft(annotated_genome_location,annotated_species_name,error_exon):
    location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment'
    list_of_files_to_run_mafft_on = os.listdir(location)
    for file in list_of_files_to_run_mafft_on:
        if file.endswith(".fa"):
            command = f'"mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
            # print(command)
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            # os.system(f'{command}')
            command = f'"mafft" --localpair --clustalout --maxiterate 16 --reorder "{location}/{file}" > "{location}/alignment_clustal_{file}.txt"'
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            
            # os.system(f'{command}')
    return(location)

In [5]:
def process_mafft_output(mafft_run_folder, error_exon):
    list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
    score_output = []
    score = 99
    min_score_sequence = ''
    alignment_file = ''
    for file in list_of_files_in_mafft_run_folder:
        if file.endswith(".fa.hat2"):
            with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
                dist_matrix_list = dist_matrix_file.readlines()
            
            sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
            distance_score = float(dist_matrix_list[-1].rstrip())
            if len(score_output) < 5:
                score_output.append( [sequence_name,distance_score])
            else:
                for i in range(len(score_output)):
                    score_at_this_index = score_output[i][1]
                    if distance_score < score_at_this_index:
                        score_output[i] = [sequence_name,distance_score]
                        break
            if distance_score < score:
                score = distance_score
                min_score_sequence = sequence_name
                alignment_file = f'alignment_{file.replace(".hat2",".txt")}'
    # print(score_output)
    print(f"min = {min_score_sequence}, {score}" )
    # score_out_merged = '\n'.join(score_output)
    print(f"5 top scores:\n{score_output}")
    print(f"{mafft_run_folder}/{alignment_file}")
    
    alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
    # print (records.id)
    fasta_start_position = 0
    fasta_end_position = 0
    start_switch = 0
    end_switch = 0
    alignment_name = ''
    for records in alignment_file:
        
        
        
        if error_exon in records.id:
            gap_counter = 0
            base_counter = 0
            for current_position in range(len(records.seq)):
                sequence_length = len(records.seq) - records.seq.count('-')
                # print(f"fasta_end_position {fasta_end_position} fasta_start_position {fasta_start_position}")
                # print(f"current_position = {current_position}, {len(records.seq)}")
                # print(records.seq[current_position])
                # print("fasta_start_position",fasta_start_position)
                # print("start_switch",start_switch)
                
                # print(gap_counter, base_counter)
                # print(5,0.2*sequence_length)
                if start_switch == 1 and records.seq[current_position] == "-" and gap_counter > 3 and (len(records.seq[:current_position]) - records.seq[:current_position].count('-')) < (0.1*sequence_length):
                #  :
                    # print("\n\nhere\n\n")
                    start_switch = 0
                    gap_counter = 0

                if "-" not in records.seq[current_position] and start_switch == 0 :
                    fasta_start_position = current_position
                    start_switch = 1
                
                    
                
                    
                if end_switch == 1 and "-" not in records.seq[current_position] and (len(records.seq[current_position:]) - records.seq[current_position:].count('-')) > 0.1*sequence_length :
                    end_switch = 0
                    gap_counter = 0
                
                    
                if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                    # print(f"base_counter {base_counter}")
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    fasta_end_position = current_position
                    end_switch = 1
                if "-" in records.seq[current_position]:
                    gap_counter += 1
                else:
                    gap_counter = 0
                    base_counter += 1
        else:
            alignment_name = records.id
            if fasta_end_position == 0:
                
                fasta_end_position = current_position
            end_switch = 1
            start_switch = 1

        if start_switch == 1 and end_switch == 1:
            print(fasta_start_position, fasta_end_position)
            gene_sequence = records.seq[fasta_start_position:fasta_end_position]
            print(f"{records.id}\n{gene_sequence}")
    return(gene_sequence, alignment_name)

# process_mafft_output(mafft_run_folder, error_exon)

In [6]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False

def check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon):
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    
    list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/")
    
    if error_exon not in list_of_exon_directories:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
    elif error_exon in list_of_exon_directories:
        list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        if "for_alignment" not in list_of_folders_1:
            os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{file}")


In [7]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
    print("Getting Gene")
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp")

    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    # subprocess.run("pwd")
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)

In [8]:
def mafft_process(previous_exon_coordinates,
                  next_exon_coordinates,
                  current_exon_coordinates,
                  query_species,
                  query_transcript,
                  query_location,
                  annotated_genome_location,
                 genome_location):
    query_exon = current_exon_coordinates[6].split("query")[-1][1:]
    error_exon = query_exon
    # print("AA")
    genome_file = get_genome_file(genome_location,species)
    
    upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    
    
    with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
        query_file_list = query_file.readlines()
        query_fasta_sequence = "".join(query_file_list)
        left_overhang = query_file_list[0].split("Frame")[1][1]
        right_overhang = query_file_list[0].split("rightoh")[1][1]
        original_query_name = query_file_list[0]
        original_query = query_file_list[1]
        
    query_length = len(query_fasta_sequence.split("\n")[1])
    
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)
    # print(annotated_species_name)

    # print(annotated_genome_location)
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    # assert False
    check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon)
    
    
    make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length)
    
    return(annotated_species_name, error_exon,species, genome_file,gene_sequence,left_overhang,right_overhang,scaffold, original_query_name,original_query )


In [12]:
family_group = "4.Pierinae"
blast_output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(blast_output_location)

if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")

for species in species_list:
    annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    query_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Query"
    list_of_query_species = os.listdir(query_location)
    if "desktop.ini" in list_of_query_species:
        list_of_query_species.remove("desktop.ini")
#     list_of_query_species = ["Papilio_xuthus"]
    for query_species in list_of_query_species:
        list_of_query_transcripts = os.listdir(f"{query_location}/{query_species}")
        if "desktop.ini" in list_of_query_transcripts:
            list_of_query_transcripts.remove("desktop.ini")
        for query_transcript in list_of_query_transcripts:


            genome_location = "/mnt/f/Genomes_2023-12-26"



            output_coordinate_file = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"
            exons_to_check_list = ["Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15"]
#             exons_to_check_list = ["Exon_9"]

            list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
            coordinate_file_name = ''
        #     print(list_of_files_in_species_folder)
            for file_names in list_of_files_in_species_folder:

                if file_names.endswith("_coordinates_old.csv"):
                    coordinate_file_name = file_names
            if coordinate_file_name =='':
                print(f"Coordinate file error")
                assert False

            with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
                coordinate_file_lines = temp_file_open.readlines()
            # print(coordinate_file_lines[0])
            for exons_to_check in exons_to_check_list:

                for i in range(1,len(coordinate_file_lines)):
                    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
                    current_exon_coordinates = coordinate_file_lines[i].split(",")
                    if i == 1:
                        scaffold_first_exon = coordinate_file_lines[i].split(",")[1]
                        first_exon_start, first_exon_end = coordinate_file_lines[i].split(",")[2],coordinate_file_lines[i].split(",")[3]
                    # if coordinate_file_lines_split[5] == "Y":
                    if  coordinate_file_lines_split[6].endswith(exons_to_check):
                        if i == 1:
                            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
                            assert False
                        else:
                            print(f"Scaffold = {scaffold_first_exon}, start = {first_exon_start}, end = {first_exon_end}")
                            print(coordinate_file_lines[i])
                            # process_current_exon = input("Error Found! Process?")
                            process_current_exon = 'y'
                            if process_current_exon.lower()[0] == "y":
                                if i>2:
                                    previous_exon_number = i-2
                                else:
                                    previous_exon_number = i-1
                                if i+2 <=int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = i+2
                                else:
                                    next_exon_number = i+1
                                if i >= int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = int(exons_to_check_list[-1].split("_")[1])
#                                     continue

                                '''
                                the above code ensures that missing exons do not cause issues
                                '''

                                previous_exon_coordinates = ''
                                next_exon_coordinates = ''
                                while True:
                                    print(previous_exon_number)
                                    if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                                        previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                        break
                                    # else:
                                    #     proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                                    #     if proceed_prompt.lower()[0] == "y":
                                    #         previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                    #         break
                                    previous_exon_number -= 1
                                while True:
                                    print(next_exon_number)
                                    try:
                                        if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number]
                                            break
                                    except:
                                        if coordinate_file_lines[next_exon_number-1].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number-1]
                                            break
                                   
                                    next_exon_number += 1


                                print(previous_exon_coordinates)
                                print(next_exon_coordinates)
                                annotated_species_name, error_exon, species, genome_file, gene_sequence,left_overhang,right_overhang,scaffold,original_query_name, original_query = mafft_process(previous_exon_coordinates,
                                                                                                                                  next_exon_coordinates,
                                                                                                                                  current_exon_coordinates,
                                                                                                                                  query_species,
                                                                                                                                  query_transcript,
                                                                                                                                  query_location,
                                                                                                                                  annotated_genome_location,
                                                                                                                                     genome_location)

                                mafft_run_folder = run_mafft(annotated_genome_location,annotated_species_name,error_exon)

                                possible_gene_sequence, alignment_name = process_mafft_output(mafft_run_folder, error_exon)

                                print(f"Original query = {original_query}")
                                print(f"New query = {possible_gene_sequence}")
                                
                                list_of_folders_inside_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
                                if "for_blast" not in list_of_folders_inside_exon_folder:
                                    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                else:
                                    list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                    for file in list_of_files:
                                        os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/{file}")


                                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/new_query.txt",'w') as query_file:
                                    query = f">Query_{error_exon}_{alignment_name}\n{possible_gene_sequence}"
                                    query_file.write(query)
                    

Scaffold = FR989955.1, start = 7585053, end = 7585149
Anthocharis_cardamines,FR989955.1,7584086,7584636,1,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_0_query_Exon_2,1,180,180

1
4
Anthocharis_cardamines,FR989955.1,7585053,7585149,1,N,Pieris_brassicae_XM_045665865.1_Frame_0_rightoh_1_query_Exon_1,1,32,32

Anthocharis_cardamines,FR989955.1,7582637,7582860,1,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_0_query_Exon_4,2,70,74

Getting Gene
min = set17_frame2, 0.311
5 top scores:
[['set17_frame2', 0.311]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Anthocharis_cardamines/Period_gene_genomic_sequence_individual_exon_Pieris_brassicae/Exon_2/for_alignment/alignment_Exon_2_translated_genomic_sequence_17_frame2.fa.txt
5 188
Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_0_query_Exon_2
ALESILNNILTEDKNLRTYRRSISFGQNIKKDLIPLLLHVKDDKTIELLMKILVNLTIPVECLLSVDSISNSEVGKHTIFEINSLLSSTKAAFVDNRATKVIVEFLKKNSDFKQKTKLTLEQCANISNTLLFLRNILHIPEDVI-ISS

min = set32_frame2, 0.279
5 top scores:
[['set32_frame2', 0.279], ['set43_frame2', 1.613], ['set23_frame1', 1.713], ['set21_frame2', 1.741], ['set17_frame0', 1.649]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Anthocharis_cardamines/Period_gene_genomic_sequence_individual_exon_Pieris_brassicae/Exon_8/for_alignment/alignment_Exon_8_translated_genomic_sequence_32_frame2.fa.txt
10 43
Pieris_brassicae_XM_045665865.1_Frame_0_rightoh_2_query_Exon_8
DLIVTNHILLMFLDNIMHLPDYKGSGKMIDHLR
10 43
set32_frame2
DLIVTNHILLMFVDNVMKLPDYKGSSNMIDHIR
Original query = DLIVTNHILLMFLDNIMHLPDYKGSGKMIDHLR
New query = DLIVTNHILLMFVDNVMKLPDYKGSSNMIDHIR
Scaffold = FR989955.1, start = 7585053, end = 7585149
Anthocharis_cardamines,FR989955.1,7580339,7580537,1,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_0_query_Exon_9,1,66,66

7
11
Anthocharis_cardamines,FR989955.1,7581071,7581262,1,N,Pieris_brassicae_XM_045665865.1_Frame_0_rightoh_0_query_Exon_7,1,64,64

Anth

Scaffold = FR989955.1, start = 7585053, end = 7585149
Anthocharis_cardamines,FR989957.1,8875128,8875289,1,Y,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_15,14,46,53

13
15
16
Anthocharis_cardamines,FR989955.1,7577635,7577812,1,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_2_query_Exon_13,1,57,57

Anthocharis_cardamines,FR989955.1,7574530,7574713,1,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_0_query_Exon_16,1,60,61

Getting Gene
min = set34_frame2, 1.608
5 top scores:
[['set34_frame2', 1.608], ['set1_frame2', 1.801], ['set61_frame0', 1.841], ['set22_frame2', 1.859], ['set46_frame1', 1.868]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Anthocharis_cardamines/Period_gene_genomic_sequence_individual_exon_Pieris_brassicae/Exon_15/for_alignment/alignment_Exon_15_translated_genomic_sequence_34_frame2.fa.txt
12 70
Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_15
TIIHTESETVLEEIKHPFGIPM---EQDGELSVEVADSG

Getting Gene
min = set39_frame2, 0.363
5 top scores:
[['set39_frame2', 0.363], ['set1_frame2', 1.854], ['set18_frame1', 1.839], ['set19_frame1', 1.833], ['set51_frame2', 1.855]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Anthocharis_cardamines/Period_gene_genomic_sequence_individual_exon_Maniola_jurtina/Exon_7/for_alignment/alignment_Exon_7_translated_genomic_sequence_39_frame2.fa.txt
0 64
Maniola_jurtina_XM_045907399.1_Frame_0_rightoh_0_query_Exon_7
VVTAIREFVQAIEVYKKSAHICNNDKDELDKLQVKMCETDELRSLLVLLLRYYNPKYQSKQYLQ
0 64
set39_frame2
VVTAIREFVQAIEVYKKSIHICQKEKEVLLKLQMKMCESEELRSLLVLLLRHYNPKYHSKQYLQ
Original query = VVTAIREFVQAIEVYKKSAHICNNDKDELDKLQVKMCETDELRSLLVLLLRYYNPKYQSKQYLQ
New query = VVTAIREFVQAIEVYKKSIHICQKEKEVLLKLQMKMCESEELRSLLVLLLRHYNPKYHSKQYLQ
Scaffold = FR989955.1, start = 7585053, end = 7585149
Anthocharis_cardamines,FR989955.1,7580889,7580989,1,N,Pieris_brassicae_XM_045665865.1_Frame_0_rightoh_2_query_Exon_8,1,33,33

6
1

Getting Gene
min = set29_frame2, 0.779
5 top scores:
[['set29_frame2', 0.779], ['set1_frame2', 1.694], ['set59_frame0', 1.739], ['set69_frame0', 1.746], ['set45_frame1', 1.748]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Anthocharis_cardamines/Period_gene_genomic_sequence_individual_exon_Maniola_jurtina/Exon_14/for_alignment/alignment_Exon_14_translated_genomic_sequence_29_frame2.fa.txt
21 55
Maniola_jurtina_XM_045907399.1_Frame_1_rightoh_2_query_Exon_14
PGSSFTTTDDNVTKPSWLEVVQKSQEYKITLDLG
21 55
set29_frame2
PGSSFSSTCEQGT-PSWLQVVQKSQEFRLTLDHE
Original query = PGSSFTTTDDNVTKPSWLEVVQKSQEYKITLDLG
New query = PGSSFSSTCEQGT-PSWLQVVQKSQEFRLTLDHE
Scaffold = FR989955.1, start = 7585053, end = 7585149
Anthocharis_cardamines,FR989957.1,8875128,8875289,1,Y,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_15,14,46,53

13
15
16
Anthocharis_cardamines,FR989955.1,7577635,7577812,1,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_2_que

...

Scaffold = NC_059524.1, start = 6750840, end = 6750936
Pieris_rapae,NC_059524.1,6761661,6761835,0,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_2_query_Exon_13,1,57,57

11
15
Pieris_rapae,NC_059524.1,6759184,6759650,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_1_query_Exon_11,1,154,154

Pieris_rapae,NC_059524.1,6763328,6763489,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_15,2,53,53

Getting Gene
min = set48_frame1, 0.511
5 top scores:
[['set48_frame1', 0.511], ['set39_frame2', 1.781], ['set37_frame0', 1.751], ['set72_frame2', 1.784], ['set87_frame2', 1.809]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Pieris_rapae/Period_gene_genomic_sequence_individual_exon_Pieris_brassicae/Exon_13/for_alignment/alignment_Exon_13_translated_genomic_sequence_48_frame1.fa.txt
5 62
Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_2_query_Exon_13
ELKFSFSDISKISDLNSSKTTVRLQDQPQNMVLTVIPENIFQIDKSKHIAGIVNYTP
5 62
set48_frame1
K

Getting Gene
min = set37_frame1, 0.237
5 top scores:
[['set27_frame0', 1.901], ['set12_frame1', 1.864], ['set37_frame1', 0.237]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Pieris_rapae/Period_gene_genomic_sequence_individual_exon_Maniola_jurtina/Exon_6/for_alignment/alignment_Exon_6_translated_genomic_sequence_37_frame1.fa.txt
23 143
Maniola_jurtina_XM_045907399.1_Frame_1_rightoh_0_query_Exon_6
INVQGLSHNTPTDDDISNVLKEFTVDFLLKGYNSLVRTLHSQILTNVQLEIDTSHFFWLVTYFLKFATQIELDLEHVCCVLSFEIMSYLTAEGVNLCEQFELAIKLDGNDLKPSIRRLHL
23 143
set37_frame1
INVQGLLHNMPTDDDISNVIKEFTVDFLLKGYNSLVRTLHTQILTNVHLEIDTSHFFWLVTYFLKFATQIELDLEYVDSVLSFDIVLYLTAEGVNLCEQFELALKLDGCDLKPNIRRLHL
Original query = INVQGLSHNTPTDDDISNVLKEFTVDFLLKGYNSLVRTLHSQILTNVQLEIDTSHFFWLVTYFLKFATQIELDLEHVCCVLSFEIMSYLTAEGVNLCEQFELAIKLDGNDLKPSIRRLHL
New query = INVQGLLHNMPTDDDISNVIKEFTVDFLLKGYNSLVRTLHTQILTNVHLEIDTSHFFWLVTYFLKFATQIELDLEYVDSVLSFDIVLYLTAEGVNLCEQFELALKLDGCDLKPNIRRLHL
Scaffold = NC_

min = set48_frame1, 1.247
5 top scores:
[['set48_frame1', 1.247], ['set39_frame2', 1.742], ['set82_frame2', 1.803], ['set87_frame2', 1.815], ['set34_frame2', 1.864]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Pieris_rapae/Period_gene_genomic_sequence_individual_exon_Maniola_jurtina/Exon_13/for_alignment/alignment_Exon_13_translated_genomic_sequence_48_frame1.fa.txt
5 65
Maniola_jurtina_XM_045907399.1_Frame_2_rightoh_2_query_Exon_13
KLRFSLKDISQSSILNSAQTSSLILPELSKGNVLPPISDNFYQIHNKKQNHLASIVNFTP
5 65
set48_frame1
KLKFSLSDISNTSDLKTSKT-SVRLDDQPQDRVLKVMPKNFFQI--DKQKHIAGIVKYSP
Original query = KLRFSLKDISQSSILNSAQTSSLILPELSKGNVLPPISDNFYQIHNKKQNHLASIVNFTP
New query = KLKFSLSDISNTSDLKTSKT-SVRLDDQPQDRVLKVMPKNFFQI--DKQKHIAGIVKYSP
Scaffold = NC_059524.1, start = 6750840, end = 6750936
Pieris_rapae,NC_059524.1,6762455,6762529,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_14,1,22,24

12
15
Pieris_rapae,NC_059524.1,6760708,676090

Scaffold = NC_059524.1, start = 6750840, end = 6750936
Pieris_rapae,NC_059524.1,6755025,6755385,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_0_query_Exon_6,1,120,120

4
8
Pieris_rapae,NC_059524.1,6752985,6753208,0,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_0_query_Exon_4,3,73,74

Pieris_rapae,NC_059524.1,6757025,6757125,0,N,Pieris_brassicae_XM_045665865.1_Frame_0_rightoh_2_query_Exon_8,1,33,33

Getting Gene
min = set37_frame1, 0.274
5 top scores:
[['set27_frame0', 1.893], ['set12_frame1', 1.864], ['set37_frame1', 0.274]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Pieris_rapae/Period_gene_genomic_sequence_individual_exon_Bicyclus_anynana/Exon_6/for_alignment/alignment_Exon_6_translated_genomic_sequence_37_frame1.fa.txt
23 143
Bicyclus_anynana_XM_052891332.1_Frame_1_rightoh_0_query_Exon_6
INVQGLSHSTPTNDDISNVLKEFTVDFLLKGYNSLVRTLHSQILTNVQLEIDTSHFFWLVTYFLKFATQIELDLEHVCWVLSYEIISYLTAEGVNLCEQFELAIKLDGNDLKPSIRRLHL
23 143
set37

Scaffold = NC_059524.1, start = 6750840, end = 6750936
Pieris_rapae,NC_059524.1,6761661,6761835,0,N,Pieris_brassicae_XM_045665865.1_Frame_2_rightoh_2_query_Exon_13,1,57,57

11
15
Pieris_rapae,NC_059524.1,6759184,6759650,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_1_query_Exon_11,1,154,154

Pieris_rapae,NC_059524.1,6763328,6763489,0,N,Pieris_brassicae_XM_045665865.1_Frame_1_rightoh_2_query_Exon_15,2,53,53

Getting Gene
min = set48_frame1, 1.34
5 top scores:
[['set48_frame1', 1.34], ['set34_frame2', 1.789], ['set46_frame0', 1.811], ['set39_frame2', 1.83], ['set82_frame2', 1.846]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/4.Pierinae/1.Blast_result/Pieris_rapae/Period_gene_genomic_sequence_individual_exon_Bicyclus_anynana/Exon_13/for_alignment/alignment_Exon_13_translated_genomic_sequence_48_frame1.fa.txt
5 65
Bicyclus_anynana_XM_052891332.1_Frame_2_rightoh_2_query_Exon_13
TLKFSLNDISQSSILNNSSQTSSFILPELSKGNALSPTPDNFYQINKKQNLFASIMNFTP
5 65
set48_frame1
K